## Optimización de la planificación de quirófanos mediante algoritmos genéticos
#### Autor: Jesús García Armario
#### Trabajo Fin de Grado - Ingeniería Informática

In [5]:
# Imports necesarios
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
import UtilsGenetico as utils

In [ ]:
# Definimos al individuo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Definimos los atributos del individuo
toolbox = base.Toolbox()
toolbox.register("attr_bool", utils.randomBool)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=utils.NUMERO_CIRUGIAS)